# Create a time-of-flight lookup table for ODIN

In [ ]:
import scipp as sc
import sciline as sl
from ess.reduce import time_of_flight
from ess.odin.beamline import choppers

## Setting up the workflow

In [ ]:
source_position = sc.vector([0, 0, 0], unit='m')

wf = time_of_flight.TofLookupTableWorkflow()
wf[time_of_flight.DiskChoppers] = choppers(source_position)
wf[time_of_flight.SourcePosition] = source_position
wf[time_of_flight.NumberOfSimulatedNeutrons] = 5_000_000
wf[time_of_flight.SimulationSeed] = 1234
wf[time_of_flight.PulseStride] = 2
wf[time_of_flight.LtotalRange] = sc.scalar(55.0, unit="m"), sc.scalar(65.0, unit="m")
wf[time_of_flight.DistanceResolution] = sc.scalar(0.1, unit="m")
wf[time_of_flight.TimeResolution] = sc.scalar(250.0, unit='us')
wf[time_of_flight.LookupTableRelativeErrorThreshold] = 0.02

## Compute the table

In [ ]:
table = wf.compute(time_of_flight.TimeOfFlightLookupTable)
table

In [ ]:
table.plot()

## Save to file

In [ ]:
# Save chopper metadata
# TODO: storing the choppers as a PyObject is skipped when saving to disk
table.coords['choppers'] = sc.scalar(disk_choppers)
# Write to file
table.save_hdf5('ODIN-tof-lookup-table.h5')